## MCP 工具调用

https://github.com/langchain-ai/langchain-mcp-adapters  
pip install langchain-mcp-adapters

# Quickstart
## Client

In [ ]:
# Create server parameters for stdio connection
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

# 配置mcp服务参数，Python代码函数
server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your math_server.py file
    args=["/Users/lzc/TNTprojectZ/LangChainStudy/math_server.py"],
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize the connection
        await session.initialize()

        # Get tools
        tools = await load_mcp_tools(session)

        # Create and run the agent
        llm = ChatOpenAI(
            model="Qwen/Qwen3-8B",
            # model="Qwen/Qwen3-30B-A3B-Instruct-2507",
            api_key="sk-jvjyawqpodlkxlywatvemcdykkrbvthhjyjyapyvtnifwlbl",
            base_url="https://api.siliconflow.cn/v1/",
        )
        agent = create_react_agent(llm, tools)
        agent_response = await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})

In [2]:
agent_response

{'messages': [HumanMessage(content="what's (3 + 5) x 12?", additional_kwargs={}, response_metadata={}, id='ec97392c-d101-4cb8-9ab0-351b2521efcb'),
  AIMessage(content='\n\n', additional_kwargs={'tool_calls': [{'id': '0198dfb1b9edf9f0b894133ac8072227', 'function': {'arguments': ' {"a": 3, "b": 5}', 'name': 'add'}, 'type': 'function', 'index': 0}, {'id': '0198dfb1bc1ca9c2dd22bffde46de48e', 'function': {'arguments': ' {"a": 8, "b": 12}', 'name': 'multiply'}, 'type': 'function', 'index': 1}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 535, 'prompt_tokens': 239, 'total_tokens': 774, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 507, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '0198dfb172b64ff27c340c41d646b53f', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7e448b5b-76d8-44c8-9a4

# Multiple MCP Servers
## Server
### math_server.py
### weather_server.py
## Client

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

client = MultiServerMCPClient(
    {
        "math": {
            "command": "python",
            # Make sure to update to the full absolute path to your math_server.py file
            "args": ["/Users/lzc/TNTprojectZ/LangChainStudy/math_server.py"],
            "transport": "stdio",
        },
        "weather": {
            # 命令行运行python weather_server.py
            # Make sure you start your weather server on port 8000
            "url": "http://localhost:8000/mcp/",
            "transport": "streamable_http",
        }
    }
)
tools = await client.get_tools()
llm = ChatOpenAI(
        model="Qwen/Qwen3-8B",
        # model="Qwen/Qwen3-30B-A3B-Instruct-2507",
        api_key="sk-jvjyawqpodlkxlywatvemcdykkrbvthhjyjyapyvtnifwlbl",
        base_url="https://api.siliconflow.cn/v1/",
    )
agent = create_react_agent(llm, tools)
math_response = await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})
weather_response = await agent.ainvoke({"messages": "what is the weather in nyc?"})

In [2]:
math_response

{'messages': [HumanMessage(content="what's (3 + 5) x 12?", additional_kwargs={}, response_metadata={}, id='38325d60-a373-48fb-acba-454aa8a128e3'),
  AIMessage(content='\n\n', additional_kwargs={'tool_calls': [{'id': '0198e04d3b01768d8d9310d7860d9ebf', 'function': {'arguments': ' {"a": 3, "b": 5}', 'name': 'add'}, 'type': 'function', 'index': 0}, {'id': '0198e04d3d12351ff0456ed52e41dfd9', 'function': {'arguments': ' {"a": 8, "b": 12}', 'name': 'multiply'}, 'type': 'function', 'index': 1}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 276, 'prompt_tokens': 297, 'total_tokens': 573, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 248, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '0198e04d23d0929317325df679648a65', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--3fdfa052-eb3c-4849-a11

In [3]:
weather_response

{'messages': [HumanMessage(content='what is the weather in nyc?', additional_kwargs={}, response_metadata={}, id='b2ce4d72-39a6-4300-9c30-b1280a53cb37'),
  AIMessage(content='\n\n', additional_kwargs={'tool_calls': [{'id': '0198e04d8866e3a9d56aee79d15d8924', 'function': {'arguments': ' {"location": "nyc"}', 'name': 'get_weather'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 99, 'prompt_tokens': 292, 'total_tokens': 391, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 90, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '0198e04d75ec4b046f60cec8c817ccf4', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--967b3e11-00ca-4227-88bc-c0c8ff199d4b-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'nyc'}, 'id': '0198e04d8866e3a9d56aee79d15d8924', 'ty

# Streamable HTTP

### To use it with Python MCP SDK streamablehttp_client:
```
cd examples/servers/streamable-http-stateless/  
uv run mcp-simple-streamablehttp-stateless --port 3000
```

In [1]:
# Use server from examples/servers/streamable-http-stateless/

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

from langgraph.prebuilt import create_react_agent
from langchain_mcp_adapters.tools import load_mcp_tools
from langchain_openai import ChatOpenAI

# cd examples/servers/streamable-http-stateless/  
# uv run mcp-simple-streamablehttp-stateless --port 3000
# 启动mcp服务器，地址：http://0.0.0.0:3000
async with streamablehttp_client("http://localhost:3000/mcp/") as (read, write, _):
    async with ClientSession(read, write) as session:
        # Initialize the connection
        await session.initialize()

        # Get tools
        tools = await load_mcp_tools(session)
        llm = ChatOpenAI(
            model="Qwen/Qwen3-8B",
            # model="Qwen/Qwen3-30B-A3B-Instruct-2507",
            api_key="sk-jvjyawqpodlkxlywatvemcdykkrbvthhjyjyapyvtnifwlbl",
            base_url="https://api.siliconflow.cn/v1/",
        )
        agent = create_react_agent(llm, tools)
        math_response = await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})
        weather_response = await agent.ainvoke({"messages": "what is the weather in New York?"})
        print(math_response)
        print(weather_response)

{'messages': [HumanMessage(content="what's (3 + 5) x 12?", additional_kwargs={}, response_metadata={}, id='61a03684-a05a-42e9-8c86-b1fdf9d1dc5d'), AIMessage(content='\n\n', additional_kwargs={'tool_calls': [{'id': '0198e07018033106d3f41fc2d483c3a5', 'function': {'arguments': ' {"a": 3, "b": 5}', 'name': 'add'}, 'type': 'function', 'index': 0}, {'id': '0198e0701b82b448d78eb96a1eb199d1', 'function': {'arguments': ' {"a": 8, "b": 12}', 'name': 'multiply'}, 'type': 'function', 'index': 1}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 433, 'prompt_tokens': 344, 'total_tokens': 777, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 405, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '0198e06fde713c011837cbe398d568e0', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--05516e1d-091a-45a8-8f3f-

{'messages': [HumanMessage(content='what is the weather in New York?', additional_kwargs={}, response_metadata={}, id='1f1e351f-e6e8-4a90-ba65-732a748ae636'), AIMessage(content='\n\n', additional_kwargs={'tool_calls': [{'id': '0198e0705ad5b88ca15cd1d60f3444b1', 'function': {'arguments': ' {"location": "New York"}', 'name': 'get_weather'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 95, 'prompt_tokens': 339, 'total_tokens': 434, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 86, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '0198e07048b796ddd4ffbc68d3088b63', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d09ee855-f5bf-436a-8751-7b4ff29b76fa-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'New York'}, 'id': '0198e0705ad5b88ca15cd1d60f3444b1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 339, 'output_tokens': 95, 'total_tokens': 434, 'input_token_details': {}, 'output_token_details': {'reasoning': 86}}), ToolMessage(content="It's always sunny in New York.", name='get_weather', id='64d22bd6-e39e-40c2-8fcb-04bc288a053e', tool_call_id='0198e0705ad5b88ca15cd1d60f3444b1'), AIMessage(content='\n\nThe weather in New York is sunny.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 223, 'prompt_tokens': 380, 'total_tokens': 603, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 214, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '0198e0705dc5ff0a11313fcde69f7c5e', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--6975e4f2-cebb-41e1-8254-123e6dfbc51c-0', usage_metadata={'input_tokens': 380, 'output_tokens': 223, 'total_tokens': 603, 'input_token_details': {}, 'output_token_details': {'reasoning': 214}})]}

### Use it with MultiServerMCPClient:

In [7]:
# Use server from examples/servers/streamable-http-stateless/
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

client = MultiServerMCPClient(
    {
        "math": {
            "transport": "streamable_http",
            "url": "http://localhost:3000/mcp/"
        },
    }
)
tools = await client.get_tools()
agent = create_react_agent(llm, tools)
math_response = await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})
weather_response = await agent.ainvoke({"messages": "what is the weather in New York?"})
for item in math_response['messages']:
    print(item)
for item in weather_response['messages']:
    print(item)

content="what's (3 + 5) x 12?" additional_kwargs={} response_metadata={} id='672368aa-f52d-4a72-be86-0632e1db45af'
content='\n\n' additional_kwargs={'tool_calls': [{'id': '0198e0758855fed7ecfa36472a830362', 'function': {'arguments': ' {"a": 3, "b": 5}', 'name': 'add'}, 'type': 'function', 'index': 0}, {'id': '0198e0758a681edb778707f0b2773440', 'function': {'arguments': ' {"a": 8, "b": 12}', 'name': 'multiply'}, 'type': 'function', 'index': 1}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 333, 'prompt_tokens': 344, 'total_tokens': 677, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 305, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '0198e0756c4cba5dc82e1b3f0aeb5258', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run--6af1960c-0f50-44f2-9408-5e48e9fd1678-0' tool_calls=[{'name': 'add', '

### Passing runtime headers
连接到 MCP 服务器时，您可以使用连接配置中的标头字段包含自定义标头（例如，用于身份验证或跟踪）。以下传输支持此功能：
- sse
- streamable_http  

只有 sse 和 streamable_http 传输支持运行时标头。这些标头随每个 HTTP 请求传递到 MCP 服务器。


In [9]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

client = MultiServerMCPClient(
    {
        "weather": {
            "transport": "streamable_http",
            "url": "http://localhost:3000/mcp",
            "headers": {
                "Authorization": "Bearer YOUR_TOKEN",
                "X-Custom-Header": "custom-value"
            },
        }
    }
)
tools = await client.get_tools()
agent = create_react_agent(llm, tools)
response = await agent.ainvoke({"messages": "what is the weather in nyc?"})
for item in response['messages']:
    print(item)

content='what is the weather in nyc?' additional_kwargs={} response_metadata={} id='c5974a23-67a6-4e2e-a531-0a834de6493f'
content='\n\n' additional_kwargs={'tool_calls': [{'id': '0198e078626f9ba9d11f927c34b64931', 'function': {'arguments': ' {"location": "nyc"}', 'name': 'get_weather'}, 'type': 'function', 'index': 0}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 131, 'prompt_tokens': 339, 'total_tokens': 470, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 122, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '0198e0784a2513e35ed64c64c3f404c6', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run--ab08f77b-1861-43e4-a4c2-cff6d421c02b-0' tool_calls=[{'name': 'get_weather', 'args': {'location': 'nyc'}, 'id': '0198e078626f9ba9d11f927c34b64931', 'type': 'tool_call'}] usage_metadata={'input_toke

# Using with LangGraph StateGraph

In [1]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.graph import StateGraph, MessagesState, START
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import ToolNode, tools_condition

model = ChatOpenAI(
            model="Qwen/Qwen3-8B",
            # model="Qwen/Qwen3-30B-A3B-Instruct-2507",
            api_key="sk-jvjyawqpodlkxlywatvemcdykkrbvthhjyjyapyvtnifwlbl",
            base_url="https://api.siliconflow.cn/v1/",
        )

client = MultiServerMCPClient(
    {
        # "math": {
        #     "command": "python",
        #     # Make sure to update to the full absolute path to your math_server.py file
        #     "args": ["/Users/lzc/TNTprojectZ/LangChainStudy/examples/math_server.py"],
        #     "transport": "stdio",
        # },
        # streamable_http版和Python代码函数重复。
        "math": {
            "url": "http://localhost:3000/mcp/",
            "transport": "streamable_http",
        },
        "weather": {
            # make sure you start your weather server on port 8000
            "url": "http://localhost:3000/mcp/",
            "transport": "streamable_http",
        }
    }
)
tools = await client.get_tools()

def call_model(state: MessagesState):
    response = model.bind_tools(tools).invoke(state["messages"])
    return {"messages": response}

builder = StateGraph(MessagesState)
builder.add_node(call_model)
builder.add_node(ToolNode(tools))
builder.add_edge(START, "call_model")
builder.add_conditional_edges(
    "call_model",
    tools_condition,
)
builder.add_edge("tools", "call_model")
graph = builder.compile()
math_response = await graph.ainvoke({"messages": "what's (3 + 5) x 12?"})
weather_response = await graph.ainvoke({"messages": "what is the weather in nyc?"})

print(math_response)
print(weather_response)

{'messages': [HumanMessage(content="what's (3 + 5) x 12?", additional_kwargs={}, response_metadata={}, id='34ce90d0-9643-4e70-bf80-6b7575022006'), AIMessage(content='\n\n', additional_kwargs={'tool_calls': [{'id': '0198e07f6fbd02546207d8b46c2f97ca', 'function': {'arguments': ' {"a": 3, "b": 5}', 'name': 'add'}, 'type': 'function', 'index': 0}, {'id': '0198e07f7404cee9a0ce13f4e14d54c3', 'function': {'arguments': ' {"a": 8, "b": 12}', 'name': 'multiply'}, 'type': 'function', 'index': 1}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 748, 'prompt_tokens': 585, 'total_tokens': 1333, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 720, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '0198e07f0072bbc86f123e3ad3594988', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b7ee1150-6df5-49cf-b204